In [ ]:
%%sh
pip -q install spaCy
python -m spacy download en
python -m spacy validate

## Inspect and processing data manually

In [ ]:
%%sh
aws s3 cp s3://amazon-reviews-pds/tsv/amazon_reviews_us_Camera_v1_00.tsv.gz /tmp

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('/tmp/amazon_reviews_us_Camera_v1_00.tsv.gz', sep='\t', compression='gzip',
                   error_bad_lines=False, dtype='str')

In [ ]:
data.head()

In [ ]:
data = data.dropna()
print(data.shape)

In [ ]:
data = data[:100000]

In [ ]:
data = data[['review_body']]

In [ ]:
data.head()

In [ ]:
import spacy

spacy_nlp = spacy.load('en')

def tokenize(text):
    tokens = spacy_nlp.tokenizer(text)
    tokens = [ t.text for t in tokens ]
    return " ".join(tokens).lower()

In [ ]:
%%time
data['review_body'] = data['review_body'].apply(tokenize)

In [ ]:
data.head()

In [ ]:
import numpy as np

np.savetxt('/tmp/training.txt', data.values, fmt='%s')

In [ ]:
!head -5 /tmp/training.txt

## Training

In [ ]:
import boto3
import sagemaker

print(sagemaker.__version__)

session = sagemaker.Session()
bucket = session.default_bucket()

In [ ]:
# Run this cell if you want to use the data you processed manually

prefix = 'amazon-reviews-word2vec'

s3_train_path = session.upload_data(path='/tmp/training.txt', bucket=bucket, key_prefix=prefix+'/input/train')
s3_output = 's3://{}/{}/output/'.format(bucket, prefix)

print(s3_train_path)
print(s3_output)

In [ ]:
# Run this cell if you want to use the data processed by SageMaker Processing

prefix = 'amazon-reviews-word2vec'

s3_train_path = 
s3_output = 's3://{}/{}/output/'.format(bucket, prefix)

print(s3_train_path)
print(s3_output)

In [ ]:
from sagemaker import image_uris

region = boto3.Session().region_name    
container = image_uris.retrieve('blazingtext', region)
print(container)

In [ ]:
role = sagemaker.get_execution_role()

bt = sagemaker.estimator.Estimator(container,
                                   role, 
                                   instance_count=1, 
                                   instance_type='ml.p3.2xlarge',
                                   output_path=s3_output)

In [ ]:
bt.set_hyperparameters(mode='skipgram', subwords=True)

In [ ]:
train_data = sagemaker.TrainingInput(s3_train_path, 
                      distribution='FullyReplicated', 
                      content_type='text/plain',
                      s3_data_type='S3Prefix')

s3_channels = {'train': train_data}

In [ ]:
bt.fit(inputs=s3_channels)

In [ ]:
%%bash -s "$s3_output"
aws s3 ls --recursive $1